In [6]:
import sys
sys.path.append('/home/users/chingosa/VolcanoesHackathon/')  ### Change this
import Funcs as func
import numpy as np
import xarray as xr
import pandas as pd
import os
from pyesgf.search import SearchConnection
from concurrent.futures import ThreadPoolExecutor

conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True) 
os.environ["ESGF_PYCLIENT_NO_FACETS_STAR_WARNING"] = "on"

from importlib import reload
reload(func)

<module 'Funcs' from '/home/users/chingosa/COASt_Hackathon_4_25/Funcs.py'>

### Bulk model access for Coast Hackathon
#### Requirements:
- Requires funcs.py file
- Requires FPs subDir for storing file path CSVs
- Requires TempData subDir for storing netCDFs
---

#### Finding Files:
Allows you to search for files from a single model w/ multiple variables (other stuff probably won't need to be tweaked)


In [2]:
## Setting Up the search
## You could Imagine Looping through this tho
model = 'GFDL-ESM4'
variables = ['tas','rsdt']
tStep = 'mon'
period = 'historical'

In [3]:
## Preform the search (~75 Seconds) - Worth Saving Results
df = func.searchFPs(conn, model, variables, tStep=tStep, period=period)
df = func.addMetaData(df)

print(f'Found {len(df)} file paths for {len(df.filename.unique())} file paths for {model} with vars: {variables}')

df_VaientPicked = func.pickVarientandFilter(df)


## Varient filted File Paths
df_VaientPicked.to_csv(f'FPs/FPs_{model}.csv', index = None)
df_VaientPicked.head()

Found 32 file paths for 8 file paths for GFDL-ESM4 with vars: ['tas', 'rsdt']


,model,filename,download_url,opendap_url,Varient,period,Var,grid,timeStep,start,stop,r,i,p,f
0,GFDL-ESM4,rsdt_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_18...,http://esgf-data1.llnl.gov/thredds/fileServer/...,http://esgf-data1.llnl.gov/thredds/dodsC/css03...,r1i1p1f1,historical,rsdt,gr1,mon,1850,1949,1,1,1,1
1,GFDL-ESM4,rsdt_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_19...,http://esgf-data1.llnl.gov/thredds/fileServer/...,http://esgf-data1.llnl.gov/thredds/dodsC/css03...,r1i1p1f1,historical,rsdt,gr1,mon,1950,2014,1,1,1,1
2,GFDL-ESM4,tas_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_185...,http://aims3.llnl.gov/thredds/fileServer/css03...,http://aims3.llnl.gov/thredds/dodsC/css03_data...,r1i1p1f1,historical,tas,gr1,mon,1850,1949,1,1,1,1
3,GFDL-ESM4,tas_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_195...,http://aims3.llnl.gov/thredds/fileServer/css03...,http://aims3.llnl.gov/thredds/dodsC/css03_data...,r1i1p1f1,historical,tas,gr1,mon,1950,2014,1,1,1,1
4,GFDL-ESM4,rsdt_Amon_GFDL-ESM4_historical_r1i1p1f1_gr1_18...,http://esgf-data04.diasjp.net/thredds/fileServ...,http://esgf-data04.diasjp.net/thredds/dodsC/es...,r1i1p1f1,historical,rsdt,gr1,mon,1850,1949,1,1,1,1


---
#### Downloading / PreProcessing Files:

In [7]:
# models = ['INM-CM5-0','NorESM2-MM', 'HadGEM3-GC31-LL', 'INM-CM4-8', 'UKESM1-0-LL']
models = ['GFDL-ESM4']
df = pd.concat([pd.read_csv(f'FPs/FPs_{model}.csv', index_col=None) for model in models])

func.preprocess_and_download(models, df)



Writing to TempData/GFDL-ESM4_historical_tas_processed.nc
[############                            ] | 31% Completed | 3.70 smsWriting to TempData/GFDL-ESM4_historical_rsdt_processed.nc
[########################################] | 100% Completed | 39.50 s
[########################################] | 100% Completed | 60.61 s


---
#### Checking Processed File Size

In [8]:
def sizeof_fmt(num, suffix="B"):
    for unit in ("", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"):
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"
size = sizeof_fmt(os.path.getsize(f'TempData/{model}_historical_tas_processed.nc'))
print(f'{model} tas is: {size}')


GFDL-ESM4 tas is: 240.8MiB
